In [1]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

def initalize_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(chrome_options=chrome_options)
    return driver

driver = initalize_driver()
base_url = "https://www.personality-database.com/"

def get_page_contents(driver, url, sleep=0):
    driver.get(url)
    time.sleep(sleep)
    return driver.page_source

def get_category_urls(driver, base_url, sleep):
    page = get_page_contents(driver, base_url, sleep)
    soup = BeautifulSoup(page, 'html.parser')
    a = soup.find("div", {"id": "categories"})
    a1 = str(a).split("Real Profiles")[-1].split("Fictional Profiles")[0]
    a1 = BeautifulSoup(a1, 'html.parser')
    a1 = a1.find_all("a")
    a1 = list(map(lambda x: re.findall(r'"/(.*?)"', str(x)), a1))
    cats = [base_url + c[0] for c in a1]
    return cats

def get_subcategory_urls(driver, category_url, sleep):
    page = get_page_contents(driver, category_url, sleep)    
    soup = BeautifulSoup(page)
    a = soup.find_all("div", {"id" : re.compile(re.compile('^subcat-item.*'))})
    a1 = list(map(lambda x: x.find_all("a"), a))
    a1 = list(map(lambda x: re.findall(r'"/(.*?)"', str(x)), a1))    
    sub_cats = [base_url + c[0] for c in a1]    
    return sub_cats

def get_profiles_urls(driver, subcategory_url, sleep):
    page = get_page_contents(driver, subcategory_url, sleep)    
    soup = BeautifulSoup(page)
    a = soup.find_all("span", {"class" : re.compile(re.compile('^profile-container.*'))})
    a1 = list(map(lambda x: x.find_all("a"), a))
    a1 = list(map(lambda x: re.findall(r'"/(.*?)"', str(x)), a1))    
    profiles = [base_url + c[0] for c in a1]    
    return profiles

ModuleNotFoundError: No module named 'selenium'

In [3]:
# categories = get_category_urls(driver, base_url, 2)
# sub_cats = get_subcategories(driver, categories[1])

In [4]:
# print(len(categories))
# categories

In [5]:
def make_categories_df(driver, base_url):
    driver.quit()
    driver = initalize_driver() 
    cats = get_category_urls(driver, base_url, 1)
    categories, subcategories = [], []
    for c in cats:
#         driver.quit()
#         driver = initalize_driver() 
        sub_cats = get_subcategory_urls(driver, c, sleep=2)
        categories += [c]*len(sub_cats)
        subcategories += sub_cats
        print("category:", c, "num subcategories:", len(sub_cats))
    df = pd.DataFrame({"category":categories,"subcategory":subcategories})
    return df

# categories_df = make_categories_df(driver, base_url)
# categories_df.to_csv("data/categories.csv", index=False)
categories_df = pd.read_csv("data/categories.csv")
categories_df.head()

,category,subcategory
0,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
1,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
2,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
3,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
4,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...


In [6]:
print(len(categories_df["category"].drop_duplicates()))
for c in categories_df["category"].drop_duplicates():
    print(c)

11
https://www.personality-database.com/category/1/Pop-Culture/mbti-personality-type
https://www.personality-database.com/category/10/Noteworthy/mbti-personality-type
https://www.personality-database.com/category/14/Music/mbti-personality-type
https://www.personality-database.com/category/6/Political/mbti-personality-type
https://www.personality-database.com/category/5/Sports/mbti-personality-type
https://www.personality-database.com/category/16/The-Arts/mbti-personality-type
https://www.personality-database.com/category/21/Science/mbti-personality-type
https://www.personality-database.com/category/17/Business/mbti-personality-type
https://www.personality-database.com/category/18/Religion/mbti-personality-type
https://www.personality-database.com/category/22/Historical/mbti-personality-type
https://www.personality-database.com/category/15/Internet/mbti-personality-type


In [7]:
def make_profiles_df(driver, categories_df):
    driver.quit()
    driver = initalize_driver() 
    subcategories, profiles = [], []
    for sc in categories_df["subcategory"]:
        profs = get_profiles_urls(driver, sc, sleep=2)
        subcategories += [sc]*len(profs)
        profiles += profs
    df = pd.DataFrame({"subcategory":subcategories, "profile":profiles})
    df = categories_df.merge(df, on="subcategory")
    return df

# profiles_df = make_profiles_df(driver, categories_df)
# profiles_df.to_csv("data/profiles.csv", index=False)
profiles_df = pd.read_csv("data/profiles.csv")
profiles_df.head()

,category,subcategory,profile
0,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/1...
1,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/3...
2,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/2...
3,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/1...
4,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/1...


In [9]:
print(len(profiles_df))
exclude = ["mythical", "religion", "historical", "fictional", "memes"]
for e in exclude:
    profiles_df = profiles_df[~profiles_df["category"].str.lower().str.contains(e)]
    profiles_df = profiles_df[~profiles_df["subcategory"].str.lower().str.contains(e)]

# profiles_df.to_csv("data/profiles_filt.csv", index=False)
profiles_df = pd.read_csv("data/profiles_filt.csv")
print(len(profiles_df))
profiles_df.head()

9930
9930


,category,subcategory,profile
0,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/1...
1,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/3...
2,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/2...
3,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/1...
4,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/1...


In [10]:
# TODO the first three categories are getting dropped (who knows what else). Losing ~50% of people.

In [12]:
def parse_profile(driver, profile_url, sleep):
    page = get_page_contents(driver, profile_url, sleep)    
    soup = BeautifulSoup(page)
    votes = soup.find_all("div", {"class" : re.compile(re.compile('.*vote-breakdown-container.*'))})
    votes = [v.find_all("p") for v in votes]    
    labels = ["votes_"+re.findall(r'(?<=\<p\>)(.*)(?=\()', str(v))[0].strip() for v in votes]
    scores = [re.findall(r'(?<=\()(.*)(?=\))', str(v))[0].strip() for v in votes]
    d = dict(zip(labels, scores))
    letters = soup.find_all("div", {"class" : re.compile(re.compile('.*breakdown-item.*'))})
    labels = [l.find("h1", {"class" : re.compile(re.compile('the-letters'))}).contents[0] for l in letters[:8]]
    scores = [l.find("h5").contents[0] for l in letters[:8]]    
    d2 = dict(zip(labels, scores))
    labels = [l.find("p").contents[0] for l in letters[8:]]
    labels2 = [l.find("h5").contents[0] for l in letters[8:]]
    d3 = dict(zip(labels, labels2))
    img = soup.find("img", {"class" : re.compile(re.compile('card-image'))})["src"]
    d4 = {"img_url" : img, "profile": profile_url}    
    name = soup.find("h2", {"id" : re.compile(re.compile('.*name-normal.*'))}).contents[0]
    d5 = {"name":name}
    return pd.DataFrame.from_dict(data={**d, **d2, **d3, **d4, **d5}, orient="index").transpose()


start = 6000
finish = 8000
ps = []
driver.quit()
driver = initalize_driver()  
for i,p in enumerate(profiles_df["profile"][start:finish]):
    try:
        print(start + i)
        df = parse_profile(driver, p, 2)
        ps.append(df)
    except:
        print(p)
        time.sleep(1)
          
profiles_full_df = pd.concat(ps,axis=0)
profiles_full_df.to_csv("data/profiles_full_%s_%s.csv" % (start, finish), index=False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


6000
6001
6002
6003
6004
6005
6006
6007
6008
6009
6010
6011
6012
6013
6014
6015
6016
6017
6018
6019
6020
6021
6022
6023
6024
6025
6026
6027
6028
6029
6030
6031
6032
6033
6034
6035
6036
6037
6038
6039
6040
6041
6042
6043
6044
6045
6046
6047
6048
6049
https://www.personality-database.com/profile/35141/mario-abdo-bentez-governmentlatin-america-mbti-personality-type
6050
6051
6052
6053
6054
6055
6056
6057
6058
6059
6060
6061
6062
6063
6064
6065
6066
6067
6068
6069
6070
6071
6072
6073
6074
6075
6076
6077
6078
6079
6080
6081
6082
6083
6084
6085
6086
6087
https://www.personality-database.com/profile/41844/ben-sasse-governmentusa-mbti-personality-type
6088
6089
6090
6091
6092
6093
6094
6095
6096
6097
6098
6099
6100
6101
6102
6103
6104
https://www.personality-database.com/profile/34079/devin-nunes-governmentusa-mbti-personality-type
6105
6106
6107
6108
6109
6110
6111
6112
6113
6114
6115
6116
https://www.personality-database.com/profile/42598/howard-dean-governmentusa-mbti-personality-type
6117


6597
6598
6599
6600
https://www.personality-database.com/profile/11090/franco-baresi-football-soccer-mbti-personality-type
6601
6602
6603
6604
6605
6606
6607
6608
6609
6610
6611
6612
6613
6614
https://www.personality-database.com/profile/11088/hugo-snchez-football-soccer-mbti-personality-type
6615
6616
6617
6618
6619
6620
6621
6622
6623
6624
6625
6626
6627
6628
6629
6630
6631
6632
6633
6634
6635
6636
6637
6638
6639
6640
6641
6642
6643
6644
6645
6646
6647
6648
6649
6650
6651
6652
6653
6654
6655
6656
6657
6658
6659
6660
6661
6662
https://www.personality-database.com/profile/50631/pierreemerick-aubameyang-football-soccer-mbti-personality-type
6663
6664
6665
6666
6667
6668
6669
https://www.personality-database.com/profile/47460/roberto-firmino-football-soccer-mbti-personality-type
6670
6671
6672
6673
6674
6675
6676
6677
https://www.personality-database.com/profile/8889/syamsul-arif-munip-football-soccer-mbti-personality-type
6678
6679
https://www.personality-database.com/profile/34360/thia

7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131
7132
7133
7134
7135
7136
https://www.personality-database.com/profile/49547/ludwig-mies-van-der-rohe-artists-mbti-personality-type
7137
7138
7139
7140
7141
7142
7143
7144
7145
7146
7147
7148
https://www.personality-database.com/profile/43794/norman-saunders-artists-mbti-personality-type
7149
7150
7151
7152
7153
7154
https://www.personality-database.com/profile/43852/paul-gauguin-artists-mbti-personality-type
7155
https://www.personality-database.com/profile/40161/paul-schneideresleben-artists-mbti-personality-type
7156
7157
7158
7159
7160
7161
7162
7163
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
7175
7176
7177
7178
7179
7180
https://www.personality-database.com/profile/20159/vali-meyers-artists-mbti-personality-type
7181
7182
7183
7184
7185
7186
7187
7188
7189
7190
7191
7192
7193
7194
7195
7196
7197
7198
7199
7200
7201
7202
7203
7204
7205
7206
7207
7208
7209
7210
7211
7212
7213
72

7593
7594
7595
7596
7597
7598
7599
7600
https://www.personality-database.com/profile/40578/eric-flint-writers-literature-modern-mbti-personality-type
7601
7602
https://www.personality-database.com/profile/42461/ernest-cline-writers-literature-modern-mbti-personality-type
7603
7604
7605
https://www.personality-database.com/profile/14495/etty-hillesum-writers-literature-modern-mbti-personality-type
7606
7607
7608
7609
7610
7611
7612
7613
7614
7615
7616
7617
https://www.personality-database.com/profile/35807/gene-wolfe-writers-literature-modern-mbti-personality-type
7618
7619
7620
7621
7622
7623
7624
7625
7626
7627
7628
7629
https://www.personality-database.com/profile/40573/harry-turtledove-writers-literature-modern-mbti-personality-type
7630
7631
7632
https://www.personality-database.com/profile/9961/helga-weiss-writers-literature-modern-mbti-personality-type
7633
https://www.personality-database.com/profile/6252/hermann-broch-writers-literature-modern-mbti-personality-type
7634
7635
76

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [43]:
# driver.quit()
# driver = initalize_driver()  
# x = parse_profile(driver, "https://www.personality-database.com/profile/324/benjamin-harrison-presidents-of-the-usa-mbti-personality-type")
# x

In [22]:
x.columns

Index(['votes_ISTJ', I, S, T, J, SJ, SP, NT, NF, Dom, Aux, Tert, Inf,
       'img_url', 'profile', 'name'],
      dtype='object')

In [44]:
from glob import glob

g = glob("data/profiles_full_*")
profiles_full = pd.concat([pd.read_csv(f) for f in g],axis=0)
profiles_full.head()

,Aux,Dom,E,F,I,Inf,J,N,NF,NT,...,votes_ESTJ,votes_ESTP,votes_INFJ,votes_INFP,votes_INTJ,votes_INTP,votes_ISFJ,votes_ISFP,votes_ISTJ,votes_ISTP
0,Ni,Te,100%,NaN,NaN,Fi,100%,100%,0%,100%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ne,Fi,NaN,100%,100%,Te,NaN,100%,100%,0%,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Te,Ni,NaN,NaN,100%,Se,100%,100%,0%,100%,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
3,Ti,Ne,100%,NaN,NaN,Si,NaN,100%,0%,100%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fi,Ne,100%,100%,NaN,Si,NaN,100%,100%,0%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df = profiles_full[list(filter(lambda x: "votes"in x ,profiles_full.columns))]
profiles_full['mbti'] = df.idxmax(axis=1)
profiles_full["total_votes"] = df.sum(axis=1)
profiles_full = profiles_full.merge(profiles_df, on="profile")

# TODO this is bad, need to pull name directly from profile
profiles_full.head()

,Aux,Dom,E,F,I,Inf,J,N,NF,NT,...,votes_INTJ,votes_INTP,votes_ISFJ,votes_ISFP,votes_ISTJ,votes_ISTP,mbti,total_votes,category,subcategory
0,Ni,Te,100%,NaN,NaN,Fi,100%,100%,0%,100%,...,NaN,NaN,NaN,NaN,NaN,NaN,votes_ENTJ,2.0,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
1,Ne,Fi,NaN,100%,100%,Te,NaN,100%,100%,0%,...,NaN,NaN,NaN,NaN,NaN,NaN,votes_INFP,1.0,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
2,Te,Ni,NaN,NaN,100%,Se,100%,100%,0%,100%,...,5.0,NaN,NaN,NaN,NaN,NaN,votes_INTJ,5.0,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
3,Ti,Ne,100%,NaN,NaN,Si,NaN,100%,0%,100%,...,NaN,NaN,NaN,NaN,NaN,NaN,votes_ENTP,13.0,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
4,Fi,Ne,100%,100%,NaN,Si,NaN,100%,100%,0%,...,NaN,NaN,NaN,NaN,NaN,NaN,votes_ENFP,1.0,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...


In [118]:
# for p in list(profiles_full["profile"]):
#     print(p)

print(len(profiles_full))

exclude = ["mythical", "religion", "historical", "fictional", "memes"]
for e in exclude:
    profiles_full = profiles_full[~profiles_full["category"].str.lower().str.contains(e)]
    profiles_full = profiles_full[~profiles_full["subcategory"].str.lower().str.contains(e)]

print(len(profiles_full))

profiles_full = profiles_full[profiles_full["total_votes"]>3]

print(len(profiles_full))

906
906
906


In [115]:
profiles_full["name"].tail(100)

2128             efe aydal
2131          elvis horton
2132           ethan klein
2133           ethan klein
2136         george andrew
2137      george ouzounian
2140       gregory fluhrer
2142         harris brewis
2144            hila klein
2146       hunter avallone
2149          jaclyn glenn
2157         jj mccullough
2162         joe santagato
2163           john scarce
2165           june lapine
2167         kalvin garrah
2168         kalvin garrah
2169         kalvin garrah
2170         kalvin garrah
2173         kyle kulinski
2179         mario brisson
2187           nando moura
2188          natalie wynn
2189         nicole arbour
2192          oliver thorn
2195           paulo kogos
2196         paulo miranda
2198         ram karavadra
2199           raphal lima
2205          roman abalin
               ...        
2304     lucas feuerschtte
2307          maicon kster
2311        mark fischbach
2312        mark fischbach
2315          morgan adams
2325           phil lester
2

In [122]:
# get better / more images

import requests
import random

def get_image_urls_from_query(query):
    r = requests.get("https://api.qwant.com/api/search/images",
        params={
            'count': 50,
            'q': query,
            't': 'images',
            'safesearch': 1,
            'locale': 'en_US',
            'uiv': 4
        },
        headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
        }
    )
    response = r.json().get('data').get('result').get('items')
    urls = [r.get('media') for r in response]
    return urls

In [125]:
names, images = [], []
for name in profiles_full["name"]:
    imgs = get_image_urls_from_query(name)[:3]
    names += [name]*len(imgs) 
    images += imgs

names_imgs_df = pd.DataFrame({"name":names,"image":images})
names_imgs_df.to_csv("data/names_images.csv",index=False)

ConnectionError: HTTPSConnectionPool(host='api.qwant.com', port=443): Max retries exceeded with url: /api/search/images?count=50&q=gareth+bale&t=images&safesearch=1&locale=en_US&uiv=4 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7faf4aa38be0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [124]:
names_imgs_df

,name,image
0,manny machado,https://upload.wikimedia.org/wikipedia/commons...
1,manny machado,https://upload.wikimedia.org/wikipedia/commons...
2,manny machado,https://upload.wikimedia.org/wikipedia/commons...
3,mike trout,https://upload.wikimedia.org/wikipedia/commons...
4,mike trout,https://nbcsportsworld.files.wordpress.com/201...
5,mike trout,https://upload.wikimedia.org/wikipedia/commons...
6,pedro martinez,https://thenypost.files.wordpress.com/2018/08/...
7,pedro martinez,https://ultimasnoticias.com.do/wp-content/uplo...
8,pedro martinez,http://www.piodeportes.com/wp-content/uploads/...
9,ty cobb,http://upload.wikimedia.org/wikipedia/commons/...
